In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from patchify import patchify, unpatchify

import tensorflow as tf
import keras.backend as K
from keras.models import load_model

from skimage.morphology import skeletonize
from skan import Skeleton, summarize
from skan.csr import skeleton_to_csgraph
from skan import draw
import skan
from skimage.morphology import skeletonize
import networkx as nx

from metrics import f1,iou
from backend import *

# If folder 'input' exists populate with images, if not run the next cell.

In [10]:
if os.path.exists('input') == False:
    os.mkdir('input')
    print('Please populate the input folder with the images to be processed then rerun script!')

In [11]:
# Create all required folders if not existent
create_folder('plants')
create_folder('model_refrence')

Folder 'plants' already exists.
Folder 'model_refrence' already exists.


In [3]:
# Load the models
root_segmentation_model = tf.keras.models.load_model('model_refrence/root_4_ss.h5', custom_objects={'f1': f1,'iou':iou})
patching_model = tf.keras.models.load_model('model_refrence/discontinuity_patcher.h5', custom_objects={'f1': f1,'iou':iou})
shoot_segmentation_model = tf.keras.models.load_model('model_refrence/shoot_exp_1.h5', custom_objects={'f1': f1,'iou':iou})

In [5]:
# Extract the ROI(region of interest) from the image
for filename in os.listdir('input'):
    original_image = cv2.imread('input/' + filename)
    min_x, max_x, min_y, max_y = roi_extraction_coords('input/' + filename)
    roi_extracted_image = set_outside_pixels_to_zero(original_image, min_x, max_x, min_y, max_y)
    filename = filename.replace('.tif', '.png')
    cv2.imwrite('input/' + filename, roi_extracted_image)  # Save the extracted ROI

In [4]:
# Save the model predictions (each image has its own sub-folder)
save_model_predictions('input', root_segmentation_model, patching_model, shoot_segmentation_model, padder, 'plants')

In [5]:
# Create an image to overlay the root and shoot masks (saves the image in the same folder)
overlay_root_shoot_masks('plants')

In [6]:
# Iterate through each subfolder in the 'plants' folder
for subdir, dirs, files in os.walk('plants'):
    for file in files:
        # Check if the file is a root mask image
        if file.endswith("_root_mask.png"):
            root_mask_path = os.path.join(subdir, file)
            root_mask = cv2.imread(root_mask_path, cv2.IMREAD_GRAYSCALE)  # Read the mask image in grayscale

            # Connected components labeling on the root mask
            retval, labels, stats, centroids = cv2.connectedComponentsWithStats(root_mask)

            # Create a mask to retain only regions with area above the threshold
            mask_values = [255 if (stats[label, cv2.CC_STAT_TOP] < 800 and
                                   stats[label, cv2.CC_STAT_TOP] > 300) else 0
                           for label in range(1, retval)]

            # Apply the mask values to create a new mask
            mask = np.zeros_like(root_mask)
            for label, value in zip(range(1, retval), mask_values):
                mask[labels == label] = value

            # Apply the mask to the original root mask to get the final image
            final_image = cv2.bitwise_and(root_mask, root_mask, mask=mask)
            final_image = final_image[min_x:max_x, min_y:max_y]

            # Extract and save ROIs based on the final image
            for i in range(11):
                start_col = i * final_image.shape[1] // 5
                end_col = (i + 1) * final_image.shape[1] // 5
                roi = final_image[:,start_col:end_col]
                # Save each ROI with a sequential number
                roi_filename = f"roi_{i+1}.png"  # Naming each ROI sequentially
                roi_path = os.path.join(subdir, roi_filename)
                if np.count_nonzero(roi) != 0:
                    skeletonized_roi = skeletonize(roi)
                    skeleton_graph = skan.summarize(Skeleton(skeletonized_roi))
                    num_skeletons = len(skeleton_graph['skeleton-id'].unique())
                    cv2.imwrite(roi_path, roi)
                    

KeyboardInterrupt: 

In [ ]:
for folder in os.listdir('plants'):
    measure_images_in_folder('plants/' + folder)